In [1]:
# Import necessary libraries
import sys, os
import numpy as np
import pickle
import tensorflow as tf
import multiprocessing
import glob
from tqdm import tqdm
from waymo_open_dataset.protos import scenario_pb2
from waymo_types import object_type, lane_type, road_line_type, road_edge_type, signal_state, polyline_type

# Define the path to the dataset
dataset_path = '/scratch/rksing18/datasets/waymo/scenario/training'  # Replace with your dataset path
# dataset_path = '/scratch/rksing18/datasets/waymo/scenario/testing'  # Replace with your dataset path
src_files = glob.glob(os.path.join(dataset_path, '*.tfrecord*'))
src_files.sort()


print(f"Found {len(src_files)} TFRecord files.")

# Function to parse a single TFRecord file
def parse_tfrecord(record):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(record.numpy())  # Deserialize the TFRecord string to a Scenario object
    return scenario.SerializeToString()  # Serialize the Scenario object to a string

# Wrapper function for tf.py_function
def parse_tfrecord_wrapper(record):
    serialized_scenario = tf.py_function(
        func=parse_tfrecord,
        inp=[record],
        Tout=tf.string  # Output is a serialized string
    )
    return serialized_scenario

# Create a TensorFlow dataset from the TFRecord files
raw_dataset = tf.data.TFRecordDataset(src_files, compression_type="")

# Parse the dataset using the wrapper
parsed_dataset = raw_dataset.map(parse_tfrecord_wrapper)

# Iterate through the dataset and print some information
for i, serialized_scenario in enumerate(parsed_dataset):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(serialized_scenario.numpy())  # Deserialize the string back to a Scenario object
    print(f"Scenario {i}: ID = {scenario.scenario_id}")
    if i == 5:  # Limit to 5 scenarios for demonstration
        break

2025-04-23 19:56:21.153608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-04-23 19:56:21.153644: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Found 995 TFRecord files.
Scenario 0: ID = 28fe360951cf98d6
Scenario 1: ID = 4aea29192f2b270b


2025-04-23 19:56:36.760964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-04-23 19:56:36.761196: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-04-23 19:56:36.761421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-04-23 19:56:36.761630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2025-04-23 19:56:36.761832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Scenario 2: ID = b677bf205ebd2f50
Scenario 3: ID = 8125f2fe4cfdff0b
Scenario 4: ID = adfa8900e80b9522
Scenario 5: ID = c31a6a2a9c9fe373


In [2]:
def decode_tracks_from_proto(tracks):
    track_infos = {
        'object_id': [],
        'object_type': [],
        'trajs': [],
        'valids': []  # new field
    }

    for cur_data in tracks:
        cur_traj = [np.array([
            x.center_x, x.center_y, x.center_z,
            x.velocity_x, x.velocity_y,
            x.width, x.length, x.height,
            x.heading, x.valid
        ], dtype=np.float32) for x in cur_data.states]

        cur_traj = np.stack(cur_traj, axis=0)  # (num_timestamps, 10)
        
        track_infos['object_id'].append(cur_data.id)
        track_infos['object_type'].append(object_type[cur_data.object_type])
        track_infos['trajs'].append(cur_traj[:, :9])     # everything except 'valid'
        track_infos['valids'].append(cur_traj[:, 9])     # just the 'valid' flag

    track_infos['trajs'] = np.stack(track_infos['trajs'], axis=0)   # (num_tracks, T, 9)
    track_infos['valids'] = np.stack(track_infos['valids'], axis=0) # (num_tracks, T)
    
    return track_infos

In [3]:
for i, serialized_scenario in enumerate(parsed_dataset):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(serialized_scenario.numpy())  # Deserialize the string back to a Scenario object
    print(f"\n=== Scenario {i}: ID = {scenario.scenario_id} ===")
    print(f"Number of Timestamps: {len(scenario.timestamps_seconds)}")
    print(f"Timestamps: {scenario.timestamps_seconds}")
    print(f"Number of tracks = {len(scenario.tracks)}")

    track_infos = decode_tracks_from_proto(scenario.tracks)
    pred_indices = set([pred.track_index for pred in scenario.tracks_to_predict])
    print(f"Index of track index = {track_infos['object_id']}")
    
    print(f"Ego vehicle track ID: {scenario.sdc_track_index}")

    print(f"Tracks to predict indices: {sorted(pred_indices)}")
    print(f"Number of tracks to predict: {len(pred_indices)}")
    
    print("\nSample map feature fields:", dir(scenario.map_features[0]))
    idx = track_infos["trajs"][:,:,-1] == 1.0
    print(idx)
    print(track_infos['trajs'][idx][:,:2])

    if i == 1:  # Limit to 2 scenarios for demo
        break


=== Scenario 0: ID = 28fe360951cf98d6 ===
Number of Timestamps: 91
Timestamps: [0.0, 0.10002, 0.20004, 0.30002, 0.40004, 0.50003, 0.60002, 0.70001, 0.8, 0.89999, 0.99993, 1.09992, 1.19987, 1.29985, 1.39983, 1.49978, 1.59977, 1.69972, 1.79971, 1.89967, 1.99964, 2.09962, 2.1996, 2.29959, 2.39958, 2.49956, 2.59959, 2.69958, 2.79956, 2.89954, 2.99956, 3.09955, 3.19953, 3.29956, 3.39954, 3.49956, 3.59955, 3.69955, 3.79954, 3.89952, 3.99952, 4.0995, 4.19948, 4.29947, 4.39949, 4.49947, 4.59949, 4.6995, 4.79953, 4.89951, 4.99951, 5.09953, 5.19953, 5.29952, 5.3995, 5.49953, 5.59952, 5.6995, 5.79953, 5.89955, 5.99954, 6.09956, 6.19958, 6.2996, 6.39959, 6.4996, 6.59962, 6.6996, 6.79962, 6.89964, 6.99962, 7.09961, 7.19963, 7.29962, 7.39961, 7.49959, 7.59958, 7.69958, 7.79956, 7.89951, 7.99949, 8.0995, 8.19949, 8.29947, 8.3995, 8.49949, 8.59951, 8.69949, 8.79952, 8.8995, 8.9995]
Number of tracks = 16
Index of track index = [0, 1, 2, 3, 4, 5, 6, 8, 11, 12, 13, 141, 18, 19, 21, 142]
Ego vehicle trac

In [4]:
track_infos["trajs"][track_infos['valids']==1][:,:2].shape

(2057, 2)

In [5]:
def get_polyline_dir(polyline):
    polyline_pre = np.roll(polyline, shift=1, axis=0)
    polyline_pre[0] = polyline[0]
    diff = polyline - polyline_pre
    polyline_dir = diff / np.clip(np.linalg.norm(diff, axis=-1)[:, np.newaxis], a_min=1e-6, a_max=1000000000)
    return polyline_dir

def decode_map_features_from_proto(map_features):
    map_infos = {
        'lane': [],
        'road_line': [],
        'road_edge': [],
        'stop_sign': [],
        'crosswalk': [],
        'speed_bump': []
    }
    polylines = []

    point_cnt = 0
    for i, cur_data in enumerate(map_features):
        cur_info = {'id': cur_data.id}

        if cur_data.HasField("lane"):
            cur_info['speed_limit_mph'] = cur_data.lane.speed_limit_mph
            cur_info['type'] = lane_type[cur_data.lane.type]

            cur_info['interpolating'] = cur_data.lane.interpolating
            cur_info['entry_lanes'] = list(cur_data.lane.entry_lanes)
            cur_info['exit_lanes'] = list(cur_data.lane.exit_lanes)

            cur_info['left_boundary'] = [{
                    'start_index': x.lane_start_index, 'end_index': x.lane_end_index,
                    'feature_id': x.boundary_feature_id,
                    'boundary_type': x.boundary_type
                } for x in cur_data.lane.left_boundaries
            ]
            cur_info['right_boundary'] = [{
                    'start_index': x.lane_start_index, 'end_index': x.lane_end_index,
                    'feature_id': x.boundary_feature_id,
                    'boundary_type': road_line_type[x.boundary_type]
                } for x in cur_data.lane.right_boundaries
            ]

            global_type = polyline_type[cur_info['type']]
            cur_polyline = np.stack([np.array([point.x, point.y, point.z, global_type]) for point in cur_data.lane.polyline], axis=0)
            cur_polyline_dir = get_polyline_dir(cur_polyline[:, 0:3])
            cur_polyline = np.concatenate((cur_polyline[:, 0:3], cur_polyline_dir, cur_polyline[:, 3:]), axis=-1)

            map_infos['lane'].append(cur_info)

        elif cur_data.HasField("road_line"):
            cur_info['type'] = road_line_type[cur_data.road_line.type]

            global_type = polyline_type[cur_info['type']]
            cur_polyline = np.stack([np.array([point.x, point.y, point.z, global_type]) for point in cur_data.road_line.polyline], axis=0)
            cur_polyline_dir = get_polyline_dir(cur_polyline[:, 0:3])
            cur_polyline = np.concatenate((cur_polyline[:, 0:3], cur_polyline_dir, cur_polyline[:, 3:]), axis=-1)

            map_infos['road_line'].append(cur_info)

        elif cur_data.HasField("road_edge"):
            cur_info['type'] = road_edge_type[cur_data.road_edge.type]

            global_type = polyline_type[cur_info['type']]
            cur_polyline = np.stack([np.array([point.x, point.y, point.z, global_type]) for point in cur_data.road_edge.polyline], axis=0)
            cur_polyline_dir = get_polyline_dir(cur_polyline[:, 0:3])
            cur_polyline = np.concatenate((cur_polyline[:, 0:3], cur_polyline_dir, cur_polyline[:, 3:]), axis=-1)

            map_infos['road_edge'].append(cur_info)

        elif cur_data.HasField("stop_sign"):
            cur_info['lane_ids'] = list(cur_data.stop_sign.lane)
            point = cur_data.stop_sign.position
            cur_info['position'] = np.array([point.x, point.y, point.z])

            global_type = polyline_type['TYPE_STOP_SIGN']
            cur_polyline = np.array([point.x, point.y, point.z, 0, 0, 0, global_type]).reshape(1, 7)

            map_infos['stop_sign'].append(cur_info)

        elif cur_data.HasField("crosswalk"):
            global_type = polyline_type['TYPE_CROSSWALK']
            cur_polyline = np.stack([np.array([point.x, point.y, point.z, global_type]) for point in cur_data.crosswalk.polygon], axis=0)
            cur_polyline_dir = get_polyline_dir(cur_polyline[:, 0:3])
            cur_polyline = np.concatenate((cur_polyline[:, 0:3], cur_polyline_dir, cur_polyline[:, 3:]), axis=-1)

            map_infos['crosswalk'].append(cur_info)

        elif cur_data.HasField("speed_bump"):
            global_type = polyline_type['TYPE_SPEED_BUMP']
            cur_polyline = np.stack([np.array([point.x, point.y, point.z, global_type]) for point in cur_data.speed_bump.polygon], axis=0)
            cur_polyline_dir = get_polyline_dir(cur_polyline[:, 0:3])
            cur_polyline = np.concatenate((cur_polyline[:, 0:3], cur_polyline_dir, cur_polyline[:, 3:]), axis=-1)

            map_infos['speed_bump'].append(cur_info)

        else:
            continue
            print(f"Unexpected map feature type: {cur_data}")
            for attr in dir(cur_data):
                if attr.startswith("__"):
                    continue
                try:
                    value = getattr(cur_data, attr)
                    if not callable(value):
                        print(f"{attr} = {value}")
                except Exception as e:
                    print(f"{attr} = <error accessing attribute: {e}>")
            continue  # Skip the unexpected feature instead of raising an error

        polylines.append(cur_polyline)
        cur_info['polyline_index'] = (point_cnt, point_cnt + len(cur_polyline))
        point_cnt += len(cur_polyline)

    try:
        polylines = np.concatenate(polylines, axis=0).astype(np.float32)
    except:
        polylines = np.zeros((0, 7), dtype=np.float32)
        print('Empty polylines: ')
    map_infos['all_polylines'] = polylines
    return map_infos

In [4]:
import matplotlib.pyplot as plt

def plot_map_features(map_infos, save_path="map.png"):
    color_map = {
        'lane': 'blue',
        'road_line': 'orange',
        'road_edge': 'green',
        'stop_sign': 'red',
        'crosswalk': 'purple',
        'speed_bump': 'brown'
    }

    plt.figure(figsize=(12, 12))
    ax = plt.gca()

    all_polylines = map_infos.get("all_polylines", None)
    if all_polylines is None:
        print("No polylines found.")
        return

    for feature_type in color_map.keys():
        for item in map_infos[feature_type]:
            start, end = item['polyline_index']
            polyline = all_polylines[start:end]
            x = polyline[:, 0]
            y = polyline[:, 1]
            ax.plot(x, y, color=color_map[feature_type], linewidth=1.5, label=feature_type if feature_type not in ax.get_legend_handles_labels()[1] else "")

    ax.set_aspect('equal')
    ax.set_title("Waymo Map Features")
    ax.set_xlabel("X (meters)")
    ax.set_ylabel("Y (meters)")
    ax.grid(False)
    # ax.legend()
    plt.savefig(save_path)
    plt.close()
    print(f"Map saved to {save_path}")

In [5]:
import numpy as np
import cv2

def plot_map_features(map_infos, save_path="map.png", meta_path="meta.txt"):
    color_map = {
        'lane': [206, 229, 223],
        'road_line': [100, 100, 100],
        'road_edge': [0, 128, 0],
        'stop_sign': [0, 255, 255],
        'crosswalk': [226, 228, 234],
        'speed_bump': [169, 209, 232],
        'rest': [255, 240, 243]
    }

    all_polylines = map_infos.get("all_polylines", None)
    if all_polylines is None or len(all_polylines) == 0:
        print("No polylines found.")
        return

    # Calculate bounds with margin
    margin = 20
    scale = 2.0
    x = all_polylines[:, 0]
    y = all_polylines[:, 1]
    x_min, x_max = np.round(x.min() - margin), np.round(x.max() + margin)
    y_min, y_max = np.round(y.min() - margin), np.round(y.max() + margin)
    x_size = int(np.round((x_max - x_min) * scale))
    y_size = int(np.round((y_max - y_min) * scale))

    # Create canvas
    canvas = np.ones((y_size, x_size, 3), dtype=np.uint8) * np.array(color_map['rest'], dtype=np.uint8)

    # Draw each polyline onto the canvas
    for feature_type, color in color_map.items():
        if feature_type == 'rest':
            continue
        for item in map_infos[feature_type]:
            start, end = item['polyline_index']
            polyline = all_polylines[start:end]
            pts = np.round((polyline[:, :2] - [x_min, y_min]) * scale).astype(int)
            for i in range(len(pts) - 1):
                cv2.line(canvas, tuple(pts[i]), tuple(pts[i + 1]), color=color, thickness=1)

    # Save metadata
    meta = np.array([x_min, y_min, scale])
    np.savetxt(meta_path, meta, fmt='%.2f')

    # Save image
    cv2.imwrite(save_path, canvas)
    print(f"Map saved to {save_path}")

In [25]:
import numpy as np
import cv2

def plot_map_features(map_infos, save_path="map.png", meta_path="meta.txt"):
    color_map = {
        'lane': [200, 225, 219],
        'road_line': [161, 202, 219],         # lane_divider
        'road_edge': [255, 251, 242],         # road_divider
        'stop_sign': [0, 255, 0],           # stop_line
        'crosswalk': [116, 116, 116],         # ped_crossing
        'speed_bump': [243, 209, 53],        # walkway
        'rest': [255, 240, 243]               # background
    }

    # Set visual thickness for each type
    thickness_map = {
        'lane': 7,
        'road_line': 2,
        'road_edge': 2,
        'stop_sign': 3,
        'crosswalk': 4,
        'speed_bump': 3,
    }

    all_polylines = map_infos.get("all_polylines", None)
    if all_polylines is None or len(all_polylines) == 0:
        print("No polylines found.")
        return

    # Calculate bounds with margin
    margin = 50
    scale = 3.0
    x = all_polylines[:, 0]
    y = all_polylines[:, 1]
    x_min, x_max = np.round(x.min() - margin), np.round(x.max() + margin)
    y_min, y_max = np.round(y.min() - margin), np.round(y.max() + margin)
    x_size = int(np.round((x_max - x_min) * scale))
    y_size = int(np.round((y_max - y_min) * scale))

    # Create canvas
    canvas = np.ones((y_size, x_size, 3), dtype=np.uint8) * np.array(color_map['rest'], dtype=np.uint8)

    # Draw each polyline with individual thickness
    for feature_type, color in color_map.items():
        if feature_type == 'rest':
            continue
        thickness = thickness_map.get(feature_type, 1)
        for item in map_infos[feature_type]:
            start, end = item['polyline_index']
            polyline = all_polylines[start:end]
            pts = np.round((polyline[:, :2] - [x_min, y_min]) * scale).astype(int)
            for i in range(len(pts) - 1):
                cv2.line(canvas, tuple(pts[i]), tuple(pts[i + 1]), color=color, thickness=thickness)

    # Save metadata
    meta = np.array([x_min, y_min, scale])
    np.savetxt(meta_path, meta, fmt='%.2f')

    # Save image
    cv2.imwrite(save_path, canvas)
    print(f"Map saved to {save_path}")

In [28]:
import numpy as np
import cv2

def plot_map_features(map_infos, save_path="map.png", meta_path="meta.txt"):
    color_map = {
        'lane': [219, 225, 200],
        'road_line': [40, 40, 40],         # lane_divider
        'road_edge': [0, 0, 0],         # road_divider
        'stop_sign': [0, 0, 255],           # stop_line
        'crosswalk': [116, 116, 116],         # ped_crossing
        'speed_bump': [53, 209, 243],        # walkway
        'rest': [243, 240, 255]               # background
    }

    thickness_map = {
        'lane': 7,
        'road_line': 1,
        'road_edge': 2,
        'stop_sign': 3,
        'crosswalk': 4,
        'speed_bump': 3,
    }

    all_polylines = map_infos.get("all_polylines", None)
    if all_polylines is None or len(all_polylines) == 0:
        print("No polylines found.")
        return

    # Calculate bounds with margin
    margin = 50
    scale = 3.0
    x = all_polylines[:, 0]
    y = all_polylines[:, 1]
    x_min, x_max = np.round(x.min() - margin), np.round(x.max() + margin)
    y_min, y_max = np.round(y.min() - margin), np.round(y.max() + margin)
    x_size = int(np.round((x_max - x_min) * scale))
    y_size = int(np.round((y_max - y_min) * scale))

    canvas = np.ones((y_size, x_size, 3), dtype=np.uint8) * np.array(color_map['rest'], dtype=np.uint8)

    for feature_type, color in color_map.items():
        if feature_type == 'rest':
            continue
        thickness = thickness_map.get(feature_type, 1)
        for item in map_infos[feature_type]:
            start, end = item['polyline_index']
            polyline = all_polylines[start:end]
            pts = np.round((polyline[:, :2] - [x_min, y_min]) * scale).astype(int)
            for i in range(len(pts) - 1):
                cv2.line(canvas, tuple(pts[i]), tuple(pts[i + 1]), color=color, thickness=thickness)

    # Draw legend in top-left corner
    # legend_items = [ftype for ftype in color_map if ftype != 'rest']
    # start_y = 20
    # for i, feature in enumerate(legend_items):
    #     color = tuple(int(c) for c in color_map[feature])
    #     y = start_y + i * 25
    #     cv2.rectangle(canvas, (10, y), (30, y + 15), color, -1)
    #     cv2.putText(canvas, feature, (35, y + 13), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50, 50, 50), 1, cv2.LINE_AA)

    # Save meta and image
    meta = np.array([x_min, y_min, scale])
    np.savetxt(meta_path, meta, fmt='%.2f')
    cv2.imwrite(save_path, canvas)
    print(f"Map with legend saved to {save_path}")

In [8]:
import numpy as np
import cv2
import os

def plot_map_features(map_infos, save_path="map.png", meta_path="meta.txt"):
    color_map = {
        'lane': [219, 225, 200],
        'road_line': [40, 40, 40],
        'road_edge': [0, 0, 0],
        'stop_sign': [0, 0, 255],
        'crosswalk': [116, 116, 116],
        'speed_bump': [53, 209, 243],
        'rest': [243, 240, 255]
    }

    dark_color_map = {
        'lane': [246, 9, 0],
        'road_line': [219, 225, 200],
        # 'road_line': [20, 20, 20],
        # 'road_edge': [179, 185, 160],
        'road_edge': [20, 20, 20],
        'stop_sign': [0, 0, 255],
        'crosswalk': [116, 116, 116],
        'speed_bump': [53, 209, 243],
        'rest': [20, 20, 20]
    }

    thickness_map = {
        'lane': 12,
        'road_line': 3,
        'road_edge': 2,
        'stop_sign': 4,
        'crosswalk': 5,
        'speed_bump': 4,
    }

    all_polylines = map_infos.get("all_polylines", None)
    if all_polylines is None or len(all_polylines) == 0:
        print("No polylines found.")
        return

    # Calculate bounds with margin
    margin = 0
    scale = 4
    x = all_polylines[:, 0]
    y = all_polylines[:, 1]
    x_min, x_max = np.round(x.min() - margin), np.round(x.max() + margin)
    y_min, y_max = np.round(y.min() - margin), np.round(y.max() + margin)
    x_size = int(np.round((x_max - x_min) * scale))
    y_size = int(np.round((y_max - y_min) * scale))

    def draw_map(canvas, cmap):
        for feature_type, color in cmap.items():
            if feature_type == 'rest':
                continue
            thickness = thickness_map.get(feature_type, 1)
            for item in map_infos[feature_type]:
                start, end = item['polyline_index']
                polyline = all_polylines[start:end]
                pts = np.round((polyline[:, :2] - [x_min, y_min]) * scale).astype(int)
                for i in range(len(pts) - 1):
                    cv2.line(canvas, tuple(pts[i]), tuple(pts[i + 1]), color=color, thickness=thickness)
        return canvas
    

    # Generate visual canvas
    vis_canvas = np.ones((y_size, x_size, 3), dtype=np.uint8) * np.array(color_map['rest'], dtype=np.uint8)
    vis_canvas = draw_map(vis_canvas, color_map)

    # Generate dark canvas
    dark_canvas = np.ones((y_size, x_size, 3), dtype=np.uint8) * np.array(dark_color_map['rest'], dtype=np.uint8)
    dark_canvas = draw_map(dark_canvas, dark_color_map)

    # Save metadata
    meta = np.array([x_min, y_min, scale])
    np.savetxt(meta_path, meta, fmt='%.2f')

    # Create separate filenames
    folder, base = os.path.split(save_path)
    name = os.path.splitext(base)[0]
    vis_path = os.path.join(folder, f"vis_{name}.png")
    dark_path = os.path.join(folder, f"{name}.png")

    # Save images
    cv2.imwrite(vis_path, vis_canvas)
    cv2.imwrite(dark_path, dark_canvas)
    print(f"Saved visual map to {vis_path}")
    print(f"Saved dark map to {dark_path}")

In [6]:
import numpy as np
import cv2
import os

def plot_map_features(map_infos, xy, save_path="map.png", meta_path="meta.txt"):
    color_map = {
        'lane': [219, 225, 200],
        # 'road_line': [40, 40, 40],
        'road_edge': [0, 0, 0],
        'stop_sign': [0, 0, 255],
        'crosswalk': [116, 116, 116],
        'speed_bump': [53, 209, 243],
        'rest': [243, 240, 255]
    }

    dark_color_map = {
        'lane': [246, 9, 0],
        # 'road_line': [219, 225, 200],
        'road_edge': [20, 20, 20],
        'stop_sign': [0, 0, 255],
        'crosswalk': [116, 116, 116],
        'speed_bump': [53, 209, 243],
        'rest': [20, 20, 20]
    }

    thickness_map = {
        'lane': 12,
        'road_line': 3,
        'road_edge': 2,
        'stop_sign': 4,
        'crosswalk': 5,
        'speed_bump': 4,
    }

    all_polylines = map_infos.get("all_polylines", None)
    if all_polylines is None or len(all_polylines) == 0:
        print("No polylines found.")
        return

    # Calculate bounds with margin
    margin = 75
    scale = 3
    x = all_polylines[:, 0]
    y = all_polylines[:, 1]
    x_min, x_max = np.round(x.min() - margin), np.round(x.max() + margin)
    y_min, y_max = np.round(y.min() - margin), np.round(y.max() + margin)
    # print(f"x_min, x_max = {x_min}, {x_max}")
    # print(f"y_min, y_max = {y_min}, {y_max}")
    x_size = int(np.round((x_max - x_min) * scale))
    y_size = int(np.round((y_max - y_min) * scale))
    # print(f"x_size, y_size = {x_size}, {y_size}")
    # x_min = np.round(xy[:, 0].min() - margin)
    # x_max = np.round(xy[:, 0].max() + margin)
    # y_min = np.round(xy[:, 1].min() - margin)
    # y_max = np.round(xy[:, 1].max() + margin)
    # print(f"x_min, x_max = {x_min}, {x_max}")
    # print(f"y_min, y_max = {y_min}, {y_max}")
    # x_size = int(np.round((x_max - x_min) * scale))
    # y_size = int(np.round((y_max - y_min) * scale))
    # print(f"x_size, y_size = {x_size}, {y_size}")
       
    # Define patch_box
    x_center = x_min + 0.5 * (x_max - x_min)
    y_center = y_min + 0.5 * (y_max - y_min)
    x_size = x_max - x_min
    y_size = y_max - y_min
    patch_box = (x_center, y_center, y_size, x_size)
    # print(f"Patch Box: {patch_box}")

    # Define canvas size
    canvas_size = (np.round(scale * y_size).astype(int), np.round(scale * x_size).astype(int))
    # print(f"Canvas Size: {canvas_size}")


    
    def draw_map(canvas, cmap):
        for feature_type, color in cmap.items():
            if feature_type == 'rest':
                continue
            thickness = thickness_map.get(feature_type, 1)
            for item in map_infos[feature_type]:
                start, end = item['polyline_index']
                polyline = all_polylines[start:end]
                pts = np.round((polyline[:, :2] - [x_min, y_min]) * scale).astype(int)
                for i in range(len(pts) - 1):
                    cv2.line(canvas, tuple(pts[i]), tuple(pts[i + 1]), color=color, thickness=thickness)
        return canvas

    # Generate visual canvas
    vis_canvas = np.ones((canvas_size[0], canvas_size[1], 3), dtype=np.uint8) * np.array(color_map['rest'], dtype=np.uint8)
    vis_canvas = draw_map(vis_canvas, color_map)

    # Generate dark canvas
    dark_canvas = np.ones((canvas_size[0], canvas_size[1], 3), dtype=np.uint8) * np.array(dark_color_map['rest'], dtype=np.uint8)
    dark_canvas = draw_map(dark_canvas, dark_color_map)

    # Save metadata
    meta = np.array([x_min, y_min, scale])
    print(meta)
    np.savetxt(meta_path, meta, fmt='%.2f')

    # Create separate filenames
    folder, base = os.path.split(save_path)
    name = os.path.splitext(base)[0]
    vis_path = os.path.join(folder, f"vis_{name}.png")
    dark_path = os.path.join(folder, f"{name}.png")

    # Save images
    cv2.imwrite(vis_path, vis_canvas)
    cv2.imwrite(dark_path, dark_canvas)
    print(f"Saved visual map to {vis_path}")
    print(f"Saved dark map to {dark_path}")

In [16]:
for i, serialized_scenario in enumerate(parsed_dataset):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(serialized_scenario.numpy())  # Deserialize the string back to a Scenario object
    print(f"\n=== Scenario {i}: ID = {scenario.scenario_id} ===")
    print(f"Number of Timestamps: {len(scenario.timestamps_seconds)}")
    print(f"Timestamps: {scenario.timestamps_seconds}")
    print(f"Number of tracks = {len(scenario.tracks)}")

    track_infos = decode_tracks_from_proto(scenario.tracks)
    map_infos = decode_map_features_from_proto(scenario.map_features)
    pred_indices = set([pred.track_index for pred in scenario.tracks_to_predict])
    print(f"Index of track index = {track_infos['object_id']}")
    
    print(f"Ego vehicle track ID: {scenario.sdc_track_index}")

    print(f"Tracks to predict indices: {sorted(pred_indices)}")
    print(f"Number of tracks to predict: {len(pred_indices)}")
    
    print("\nSample map feature fields:", dir(scenario.map_features[0]))

    if i == 1:  # Limit to 2 scenarios for demo
        break


=== Scenario 0: ID = 28fe360951cf98d6 ===
Number of Timestamps: 91
Timestamps: [0.0, 0.10002, 0.20004, 0.30002, 0.40004, 0.50003, 0.60002, 0.70001, 0.8, 0.89999, 0.99993, 1.09992, 1.19987, 1.29985, 1.39983, 1.49978, 1.59977, 1.69972, 1.79971, 1.89967, 1.99964, 2.09962, 2.1996, 2.29959, 2.39958, 2.49956, 2.59959, 2.69958, 2.79956, 2.89954, 2.99956, 3.09955, 3.19953, 3.29956, 3.39954, 3.49956, 3.59955, 3.69955, 3.79954, 3.89952, 3.99952, 4.0995, 4.19948, 4.29947, 4.39949, 4.49947, 4.59949, 4.6995, 4.79953, 4.89951, 4.99951, 5.09953, 5.19953, 5.29952, 5.3995, 5.49953, 5.59952, 5.6995, 5.79953, 5.89955, 5.99954, 6.09956, 6.19958, 6.2996, 6.39959, 6.4996, 6.59962, 6.6996, 6.79962, 6.89964, 6.99962, 7.09961, 7.19963, 7.29962, 7.39961, 7.49959, 7.59958, 7.69958, 7.79956, 7.89951, 7.99949, 8.0995, 8.19949, 8.29947, 8.3995, 8.49949, 8.59951, 8.69949, 8.79952, 8.8995, 8.9995]
Number of tracks = 16
Index of track index = [0, 1, 2, 3, 4, 5, 6, 8, 11, 12, 13, 141, 18, 19, 21, 142]
Ego vehicle trac

In [19]:
import matplotlib.pyplot as plt

for i, serialized_scenario in enumerate(parsed_dataset):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(serialized_scenario.numpy())

    print(f"\n=== Scenario {i}: ID = {scenario.scenario_id} ===")
    print(f"Number of Timestamps: {len(scenario.timestamps_seconds)}")
    print(f"Number of tracks = {len(scenario.tracks)}")

    track_infos = decode_tracks_from_proto(scenario.tracks)
    pred_indices = set([pred.track_index for pred in scenario.tracks_to_predict])
    print(f"Ego vehicle track ID: {scenario.sdc_track_index}")
    print(f"Tracks to predict indices: {sorted(pred_indices)}")
    print(f"Number of tracks to predict: {len(pred_indices)}")

    plt.figure(figsize=(10, 10))
    ax = plt.gca()

    for idx, (obj_id, obj_type, traj, valid) in enumerate(zip(
        track_infos['object_id'],
        track_infos['object_type'],
        track_infos['trajs'],
        track_infos['valids'])):

        valid = valid.squeeze()  # Make sure it's (T,) shape
        x = traj[:, 0][valid == 1]  # Filter using valid mask
        y = traj[:, 1][valid == 1]

        if len(x) == 0:
            continue  # skip empty trajectories

        if idx in pred_indices:
            plt.plot(x, y, label=f"Track {idx} (predict)", linewidth=2.5, color='red')
            plt.scatter(x[-1], y[-1], c='red', marker='x')
        else:
            plt.plot(x, y, linewidth=1, color='gray', alpha=0.6)

    plt.title(f"Scenario {i} - {scenario.scenario_id}")
    plt.xlabel("X position (m)")
    plt.ylabel("Y position (m)")
    plt.axis("equal")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"temp_traj/scenario_{i}_trajectories.png")  # save the figure
    plt.close()

    print(f"Saved trajectory plot: scenario_{i}_trajectories.png")

    if i == 5:  # Stop after 2 scenarios
        break


=== Scenario 0: ID = 28fe360951cf98d6 ===
Number of Timestamps: 91
Number of tracks = 16
Ego vehicle track ID: 15
Tracks to predict indices: [0, 1, 11]
Number of tracks to predict: 3
Saved trajectory plot: scenario_0_trajectories.png

=== Scenario 1: ID = 4aea29192f2b270b ===
Number of Timestamps: 91
Number of tracks = 32
Ego vehicle track ID: 31
Tracks to predict indices: [5, 6, 7, 12, 14, 20, 21, 24]
Number of tracks to predict: 8
Saved trajectory plot: scenario_1_trajectories.png

=== Scenario 2: ID = b677bf205ebd2f50 ===
Number of Timestamps: 91
Number of tracks = 84
Ego vehicle track ID: 83
Tracks to predict indices: [6, 22]
Number of tracks to predict: 2
Saved trajectory plot: scenario_2_trajectories.png

=== Scenario 3: ID = 8125f2fe4cfdff0b ===
Number of Timestamps: 91
Number of tracks = 101
Ego vehicle track ID: 100
Tracks to predict indices: [7, 13, 23, 52, 53, 57, 83]
Number of tracks to predict: 7
Saved trajectory plot: scenario_3_trajectories.png

=== Scenario 4: ID = adf

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os

os.makedirs("temp_traj", exist_ok=True)

# Color and label maps using string object types
color_map = {
    'TYPE_VEHICLE': 'blue',
    'TYPE_PEDESTRIAN': 'green',
    'TYPE_CYCLIST': 'orange',
    'TYPE_UNSET': 'gray',
    'TYPE_OTHER': 'red',
}
color_map = {
    'Car': 'blue',
    'Pedestrian': 'green',
    'Cyclist': 'orange',
    'Unset': 'gray',
    'Other': 'red',
}
label_map = {
    'TYPE_VEHICLE': 'Vehicle',
    'TYPE_PEDESTRIAN': 'Pedestrian',
    'TYPE_CYCLIST': 'Cyclist',
    'TYPE_UNSET': 'Unset',
    'TYPE_OTHER': 'Other',
}
label_map = {
    'Car': 'Vehicle',
    'Pedestrian': 'Pedestrian',
    'Cyclist': 'Cyclist',
    'Unset': 'Unset',
    'Other': 'Other',
}


for i, serialized_scenario in enumerate(parsed_dataset):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(serialized_scenario.numpy())

    print(f"\n=== Scenario {i}: ID = {scenario.scenario_id} ===")
    print(f"Number of Timestamps: {len(scenario.timestamps_seconds)}")
    print(f"Number of tracks = {len(scenario.tracks)}")

    track_infos = decode_tracks_from_proto(scenario.tracks)
    map_infos = decode_map_features_from_proto(scenario.map_features)
    plot_map_features(map_infos, save_path=f"temp_maps/scenario_{i}_map.png", meta_path=f"temp_maps/scenario_{i}_map.txt")
    pred_indices = set([pred.track_index for pred in scenario.tracks_to_predict])
    num_frames = track_infos['trajs'].shape[1]

    fig, ax = plt.subplots(figsize=(10, 10))

    def update(frame):
        ax.clear()
        ax.set_title(f"Scenario {i} - Frame {frame}")
        ax.set_xlabel("X position (m)")
        ax.set_ylabel("Y position (m)")
        ax.axis("equal")
        ax.grid(True)

        used_labels = set()

        for idx, (traj, valid_mask, obj_type) in enumerate(zip(track_infos['trajs'], track_infos['valids'], track_infos['object_type'])):
            valid_mask = valid_mask[:frame+1].astype(bool)
            x = traj[:frame+1, 0][valid_mask]
            y = traj[:frame+1, 1][valid_mask]

            if len(x) == 0:
                continue

            color = color_map.get(obj_type, 'gray')
            label = label_map.get(obj_type, obj_type.title())

            plot_label = f"{label} (pred)" if idx in pred_indices else label
            if plot_label in used_labels:
                plot_label = None
            else:
                used_labels.add(plot_label)

            if idx in pred_indices:
                ax.plot(x, y, color=color, linewidth=2.5, label=plot_label)
                ax.scatter(x[-1], y[-1], c=color, marker='x')
            else:
                ax.plot(x, y, color=color, linewidth=1.0, alpha=0.6, label=plot_label)

        if frame == 0:
            ax.legend(loc='upper right', fontsize='small')

    ani = animation.FuncAnimation(fig, update, frames=num_frames, repeat=False)
    gif_path = f"temp_traj/scenario_{i}_trajectories.gif"
    ani.save(gif_path, writer='pillow')
    plt.close()

    print(f"Saved trajectory GIF: {gif_path}")

    if i == 5:
        break


=== Scenario 0: ID = 28fe360951cf98d6 ===
Number of Timestamps: 91
Number of tracks = 16
Unexpected map feature type: id: 362

DESCRIPTOR = <google.protobuf.pyext._message.MessageDescriptor object at 0x1554c3179af0>
Extensions = <error accessing attribute: Extensions>
_extensions_by_name = {}
_extensions_by_number = {}
crosswalk = 
id = 362
lane = 
road_edge = 
road_line = 
speed_bump = 
stop_sign = 
Unexpected map feature type: id: 363

DESCRIPTOR = <google.protobuf.pyext._message.MessageDescriptor object at 0x1554c3179af0>
Extensions = <error accessing attribute: Extensions>
_extensions_by_name = {}
_extensions_by_number = {}
crosswalk = 
id = 363
lane = 
road_edge = 
road_line = 
speed_bump = 
stop_sign = 
Unexpected map feature type: id: 364

DESCRIPTOR = <google.protobuf.pyext._message.MessageDescriptor object at 0x1554c3179af0>
Extensions = <error accessing attribute: Extensions>
_extensions_by_name = {}
_extensions_by_number = {}
crosswalk = 
id = 364
lane = 
road_edge = 
road_

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import numpy as np

os.makedirs("temp_traj", exist_ok=True)

# Color and label maps using string object types
color_map = {
    'Car': 'blue',
    'Pedestrian': 'green',
    'Cyclist': 'orange',
    'Unset': 'gray',
    'Other': 'red',
}
label_map = {
    'Car': 'Vehicle',
    'Pedestrian': 'Pedestrian',
    'Cyclist': 'Cyclist',
    'Unset': 'Unset',
    'Other': 'Other',
}

for i, serialized_scenario in enumerate(parsed_dataset):
    scenario = scenario_pb2.Scenario()
    scenario.ParseFromString(serialized_scenario.numpy())

    print(f"\n=== Scenario {i}: ID = {scenario.scenario_id} ===")
    print(f"Number of Timestamps: {len(scenario.timestamps_seconds)}")
    print(f"Number of tracks = {len(scenario.tracks)}")

    track_infos = decode_tracks_from_proto(scenario.tracks)
    map_infos = decode_map_features_from_proto(scenario.map_features)
    print(track_infos['trajs'][0])
    os.makedirs("temp_maps", exist_ok=True)
    os.makedirs("temp_traj", exist_ok=True)

    plot_map_features(map_infos, xy=track_infos["trajs"][track_infos['valids']==1][:,:2], save_path=f"temp_maps/scenario_{i}_map.png", meta_path=f"temp_maps/scenario_{i}_map.txt")
    pred_indices = set([pred.track_index for pred in scenario.tracks_to_predict])
    num_frames = track_infos['trajs'].shape[1]

    # Load background map and metadata
    map_img = plt.imread(f"temp_maps/vis_scenario_{i}_map.png")
    meta = np.loadtxt(f"temp_maps/scenario_{i}_map.txt")
    x_min, y_min, scale = meta
    height, width = map_img.shape[:2]

    # fig, ax = plt.subplots(figsize=(10, 10))
    fig = plt.figure(frameon=False)
    fig.set_size_inches(width / 100, height / 100)  # match pixel dimensions
    ax = plt.Axes(fig, [0., 0., 1., 1.])  # no padding
    ax.set_axis_off()
    fig.add_axes(ax)

    def update(frame):
        ax.clear()
        ax.imshow(map_img, extent=[x_min, x_min + width / scale, y_min + height / scale, y_min], origin='upper')
        # ax.axis('off')  # <-- hides axes, ticks, labels
        # ax.set_axis_off()

        used_labels = set()

        for idx, (traj, valid_mask, obj_type) in enumerate(zip(track_infos['trajs'], track_infos['valids'], track_infos['object_type'])):
            valid_mask = valid_mask[:frame+1].astype(bool)
            x = traj[:frame+1, 0][valid_mask]
            y = traj[:frame+1, 1][valid_mask]

            if len(x) == 0:
                continue

            color = color_map.get(obj_type, 'gray')
            label = label_map.get(obj_type, obj_type.title())

            plot_label = f"{label} (pred)" if idx in pred_indices else label
            if plot_label in used_labels:
                plot_label = None
            else:
                used_labels.add(plot_label)

            if idx in pred_indices:
                ax.plot(x, y, color=color, linewidth=2.5, label=plot_label)
                ax.scatter(x[-1], y[-1], c=color, marker='x')
            else:
                ax.plot(x, y, color=color, linewidth=1.0, alpha=0.6, label=plot_label)

        # if frame == 0:
        #     ax.legend(loc='upper right', fontsize='small')

    ani = animation.FuncAnimation(fig, update, frames=num_frames, repeat=False)
    gif_path = f"temp_traj/scenario_{i}_trajectories.gif"
    ani.save(gif_path, writer='pillow', dpi=150)
    plt.close()

    print(f"Saved trajectory GIF: {gif_path}")

    if i == 2:
        break


=== Scenario 0: ID = 28fe360951cf98d6 ===
Number of Timestamps: 91
Number of tracks = 16
[[ 1.95271594e+03  1.20463135e+03 -1.11222210e+01  3.34472656e-01
   1.09985352e+00  2.52063513e+00  5.49165869e+00  1.99127889e+00
   1.31721783e+00]
 [ 1.95274646e+03  1.20472913e+03 -1.11193485e+01  3.05175781e-01
   9.77783203e-01  2.52102232e+00  5.48462868e+00  1.99384665e+00
   1.32046151e+00]
 [ 1.95279138e+03  1.20484607e+03 -1.11173735e+01  4.49218750e-01
   1.16943359e+00  2.51716614e+00  5.45686817e+00  1.98933089e+00
   1.30919456e+00]
 [ 1.95282739e+03  1.20494849e+03 -1.11168079e+01  3.60107422e-01
   1.02416992e+00  2.51815104e+00  5.47224808e+00  1.99425101e+00
   1.29935730e+00]
 [ 1.95287305e+03  1.20506067e+03 -1.11177101e+01  4.56542969e-01
   1.12182617e+00  2.51320052e+00  5.47896719e+00  1.99760342e+00
   1.29089558e+00]
 [ 1.95291821e+03  1.20517810e+03 -1.11161785e+01  4.51660156e-01
   1.17431641e+00  2.51514530e+00  5.47461414e+00  2.00017595e+00
   1.28194368e+00]
 [ 1